https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Preprocess

In [3]:
import pandas as pd

In [5]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

In [7]:
test_df.head()

,name,class
0,../dataset/cifar/test\0_cat.png,cat
1,../dataset/cifar/test\1000_dog.png,dog
2,../dataset/cifar/test\1001_airplane.png,airplane
3,../dataset/cifar/test\1002_ship.png,ship
4,../dataset/cifar/test\1003_deer.png,deer


In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255.
)

In [10]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


## Build Model

In [11]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
    loss='categorical_crossentropy',  # Loss Function 
    metrics=['accuracy']  # Metrics / Accuracy
)  

## Training

In [14]:
batch_size

32

In [15]:
len(train_df) // 32

1562

In [16]:
len(train_generator)

1563

In [18]:
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

Epoch 1/10
1563/1563 [==============================] - 204s 130ms/step - loss: 1.7555 - accuracy: 0.3533 - val_loss: 1.4292 - val_accuracy: 0.4788
Epoch 2/10
1563/1563 [==============================] - 126s 80ms/step - loss: 1.4784 - accuracy: 0.4626 - val_loss: 1.4080 - val_accuracy: 0.4847
Epoch 3/10
1563/1563 [==============================] - 126s 81ms/step - loss: 1.3758 - accuracy: 0.5008 - val_loss: 1.2809 - val_accuracy: 0.5496
Epoch 4/10
1563/1563 [==============================] - 127s 81ms/step - loss: 1.3122 - accuracy: 0.5267 - val_loss: 1.3495 - val_accuracy: 0.5209
Epoch 5/10
1563/1563 [==============================] - 125s 80ms/step - loss: 1.2700 - accuracy: 0.5434 - val_loss: 1.3161 - val_accuracy: 0.5422
Epoch 6/10
1563/1563 [==============================] - 123s 78ms/step - loss: 1.2390 - accuracy: 0.5570 - val_loss: 1.2290 - val_accuracy: 0.5663
Epoch 7/10
1563/1563 [==============================] - 123s 79ms/step - loss: 1.2093 - accuracy: 0.5690 - val_loss: 